In [1]:
import pandas as pd

In [2]:
def load_file(path, cty_fips):
    return pd.read_csv(path).query(f'COUNTY=={cty_fips}')

In [33]:
def filter_rows(df, year_s, year_e, age_grp, init_y):
    r = df.query(f"YEAR >= {year_s} and YEAR <= {year_e} and AGEGRP=={age_grp}")
    r['YEAR'] = r['YEAR'].apply(lambda x : x - year_s + init_y)
    return r

In [34]:
df_2000 = load_file("cc-est2009-alldata-18.csv", 105)
# yrs 3-12

df_2010 = load_file("cc-est2019-alldata-18.csv", 105)
# yrs. 3-12

df_2020 = load_file("cc-est2023-alldata-18.csv", 105)
# yrs. 3-5

In [35]:
# Agegrp 0 = total
df_2000 = filter_rows(df_2000, 3, 12, 0, 2000)
df_2010 = filter_rows(df_2010, 3, 12, 0, 2010)
df_2020 = filter_rows(df_2020, 2, 5, 0, 2020)

/tmp/ipykernel_41256/92264952.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r['YEAR'] = r['YEAR'].apply(lambda x : x - year_s + init_y)
/tmp/ipykernel_41256/92264952.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r['YEAR'] = r['YEAR'].apply(lambda x : x - year_s + init_y)
/tmp/ipykernel_41256/92264952.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https

In [37]:
df_all = pd.concat([df_2000, df_2010, df_2020])


In [58]:
gender_suffix = ["_MALE", "_FEMALE"]

# Best overcounting is taking combination of all but white!
races = {
  ("NHAAC", "NHNAC"): "Non-hispanic AAPI",
  ("NHBAC",): "Black (combination) Non-Hispanic",
  ("NHIAC",): "Non-hispanic indian / alaskan native",
  ("HBAC",): "Black  Hispanic",
  ("HWA",): "White hispanic",
  ("HIAC",): "Hispanic indian alaskan",
  ("HAAC", "HNAC"): "Hispanic aapi",
  # Unknown?
  ("NHWA",): "Non-hispanic, white"
}

In [61]:
agg = pd.DataFrame()
agg['YEAR'] = df_all['YEAR']
agg['TOT_POP'] = df_all['TOT_POP']


for k in races:
  s = []
  for pre in k:
    s += [pre + g for g in gender_suffix]
    pass
  agg[races[k]] = df_all[s].sum(axis = 1)


agg['tot_man'] = agg[list(races.values())].sum(axis=1)
agg['err'] = agg['TOT_POP'] - agg['tot_man']


In [62]:
agg.to_csv("aggregate.csv")